In [4]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 759.2 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=998b571dcce6dafea48771f420704f49e31a9b7290a6bf97fb5691cb44d60395
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [5]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train_ner_df = pd.read_csv('/content/drive/MyDrive/ner_train_data.csv')
train_ner_df.head()

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


In [8]:
test_ner_df = pd.read_csv('/content/drive/MyDrive/ner_test_data.csv')
test_ner_df.head()

,Sentence,Tag
0,"라티은-원윤정, 휘닉스파크클래식 프로골퍼",PER-B EVT-B CVL-B
1,5원으로 맺어진 애인까지 돈이라는 민감한 원자재를 통해 현대인의 물질만능주의를 꼬집...,NUM-B O O O O O O O O O O O FLD-B O
2,-날로 삼키면 맛이 어떤지 일차 드셔보시겠어요 .,O O O O NUM-B O O
3,"-네, 지었습니다 .",O O O
4,◇신규 투자촉진에 방점=이번 접속료 조정결과에서 눈에 띄는 지점은 WCDMA/HSD...,O O O O O O O O TRM-B O TRM-B TRM-I ORG-B O TR...


In [9]:
# val_ner_df = pd.read_csv('/content/drive/MyDrive/ner_label.txt')
# val_ner_df.head()

In [10]:
print('학습 데이터 개수 : ', len(train_ner_df))
print('테스트 데이터 개수 : ', len(test_ner_df))

학습 데이터 개수 :  81000
테스트 데이터 개수 :  9000


In [11]:
labels = [label.strip() for label in open('/content/drive/MyDrive/ner_label.txt', 'r', encoding='utf-8')]
print('태깅 정보 : ', labels)

태깅 정보 :  ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


In [15]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
# print(train_data_sentence[0])
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
# print(test_data_sentence[0])
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

➡️ 위 코드

- ['정은' '씨를' '힘들게' '한' '가스나그,' '가만둘' '수' '없겠죠' '.'], [], [], ... 이러한 리스트가 여러 개 들어갈 수 있도록 설계해 주었다.

- ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I'], [], [], ... 이러한 리스트가 여러 개 들어갈 수 있도록 설계해 주었다.

In [16]:
print(train_data_sentence[0])
print(train_data_label[0])

['정은', '씨를', '힘들게', '한', '가스나그,', '가만둘', '수', '없겠죠', '.']
['PER-B', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [18]:
tag_size = len(tag_to_index)
print(tag_size, tag_to_index)

29 {'O': 0, 'PER-B': 1, 'PER-I': 2, 'FLD-B': 3, 'FLD-I': 4, 'AFW-B': 5, 'AFW-I': 6, 'ORG-B': 7, 'ORG-I': 8, 'LOC-B': 9, 'LOC-I': 10, 'CVL-B': 11, 'CVL-I': 12, 'DAT-B': 13, 'DAT-I': 14, 'TIM-B': 15, 'TIM-I': 16, 'NUM-B': 17, 'NUM-I': 18, 'EVT-B': 19, 'EVT-I': 20, 'ANM-B': 21, 'ANM-I': 22, 'PLT-B': 23, 'PLT-I': 24, 'MAT-B': 25, 'MAT-I': 26, 'TRM-B': 27, 'TRM-I': 28}


In [19]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [25]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer, pad_token_id_for_segment=0,
                                 pad_token_id_for_label=100):
# examples : 처리할 문장
# labels : 문장에 대한 레이블
# pad_token_id_for_segment : 세그먼트 패딩에 사용할 ID(문장 구분을 위한 패딩 토큰)
# pad_token_id_for_label : 레이블 패딩에 사용할 토큰 ID

  # 토큰 세팅
  cls_token = tokenizer.cls_token
  sep_token = tokenizer.sep_token
  pad_token_id = tokenizer.pad_token_id

  # 데이터 초기화
  input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

  # 문장과 레이블을 기준으로 반복문 실행
  for example, label in tqdm(zip(examples, labels), total=len(examples)): # 각 문장과 레이블
    tokens = []
    labels_ids = [] # 리스트 생성
    for one_word, label_token in zip(example, label): # 토큰화 split
      subword_tokens = tokenizer.tokenize(one_word) # 각 단어에 대한 토큰화 -> 서브워드 처리
      tokens.extend(subword_tokens)
      labels_ids.extend([tag_to_index[label_token]] + [pad_token_id_for_label] * (len(subword_tokens) - 1))

    # 최대 시퀸스 길이에 맞춰 자르기
    special_tokens_count = 2 # CLS, SEP 토큰
    if len(tokens) > max_seq_len - special_tokens_count:
      tokens = tokens[:(max_seq_len - special_tokens_count)]
      labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

    tokens += [sep_token]
    labels_ids += [pad_token_id_for_label]

    tokens = [cls_token] + tokens
    labels_ids = [pad_token_id_for_label] + labels_ids

    input_id = tokenizer.convert_tokens_to_ids(tokens) # 토큰을 ID로 변환
    attention_mask = [1] * len(input_id)
    padding_count = max_seq_len - len(input_id)
    input_id = input_id + ([pad_token_id] * padding_count)
    attention_mask = attention_mask + ([0] * padding_count)
    token_type_id = [pad_token_id_for_segment] * max_seq_len
    label = labels_ids + ([pad_token_id_for_label] * padding_count)

    assert len(input_id) == max_seq_len, 'Error with input length {} vs {}'.format(len(input_id), max_seq_len)
    assert len(attention_mask) == max_seq_len, 'Error with attention mask length {} vs {}'.format(len(attention_mask), max_seq_len)
    assert len(token_type_id) == max_seq_len, 'Error with token type length {} vs {}'.format(len(token_type_id), max_seq_len)
    assert len(label) == max_seq_len, 'Error with labels length {} vs {}'.format(len(label), max_seq_len)

    input_ids.append(input_id)
    attention_masks.append(attention_mask)
    token_type_ids.append(token_type_id)
    data_labels.append(label)

  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  data_labels = np.array(data_labels, dtype=np.int32)

  return (input_ids, attention_masks, token_type_ids), data_labels

In [26]:
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 81000/81000 [00:58<00:00, 1374.28it/s]


In [27]:
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 9000/9000 [00:11<00:00, 776.64it/s]


In [30]:
from transformers import TFBertForTokenClassification

In [32]:
model = TFBertForTokenClassification.from_pretrained('klue/bert-base', num_labels=tag_size, from_pt=True)
model.compile(optimizer='adam', loss=model.hf_compute_loss)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels): # 모델 이름, 분류할 클래스 수
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True) # 모델 로드
        self.classifier = tf.keras.layers.Dense(
            num_labels,
            kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02), # 최종출력
            name='classifier'
        )

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs

        # BERT의 출력값을 가져온다.
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # BERT의 마지막 숨겨진 상태를 사용
        # BERT 모델 출력의 첫 번째 요소는 각 토큰에 대한 벡터 표현
        all_output = outputs.last_hidden_state

        # 분류 레이어를 통과
        prediction = self.classifier(all_output)

        return prediction

In [38]:
def compute_loss(labels, logits): # 정답, 예측
  # 다중분류 손실함수(from_logits=True : 입력이 확률이 아닌 logit값 - 내부적으로 softmax 함수를 통해 확률로 변환)
  # reduction = tf.keras.losses.Reduction.NONE : 손실을 평균내지 않고 각 문장에 대한 손실을 개별적으로 계산
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  # 손실 마스크
  active_loss = tf.reshape(labels, (-1,)) != 100
  # 필터링
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)
  # 최종 손실 계산
  return loss_fn(labels, reduced_logits)

In [39]:
model = TFBertForTokenClassification('klue/bert-base', num_labels=tag_size)
model.compile(optimizer='adam', loss=compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [41]:
class F1score(tf.keras.callbacks.Callback):
  def __init__(self, X_test, y_test):
    self.X_test = X_test
    self.y_test = y_test

  def sequence_to_tags(self, labels_ids, pred_ids):
    label_list = []
    pred_list = []

    for i in range(0, len(labels_ids)):
      label_tag = []
      pred_tag = []

      for label_index, pred_index in zip(labels_ids[i], pred_ids[i]):
        if label_index != -100:
          label_tag.append(index_to_tag[label_index])
          pred_tag.append(index_to_tag[pred_index])

      label_list.append(label_tag)
      pred_list.append(pred_tag)

    return label_list, pred_list

  def on_epoch_end(self, epoch, logs=[]):
    y_predicted = self.model.predict(self.X_test)
    y_predicted = np.argmax(y_predicted, axis=2)

    label_list, pred_list = self.sequence_to_tags(self.y_test, y_predicted)

    score = f1_score(label_list, pred_list, suffix=True)
    print('f1 score : ', score*100)
    print(classification_report(label_list, pred_list, suffix=True))

In [42]:
f1_score_report = F1score(X_test, y_test)

In [43]:
model.fit(X_train, y_train, epochs=2, batch_size=64, callbacks=[f1_score_report])

Epoch 1/2


KeyboardInterrupt: 

In [44]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

NameError: name 'convert_examples_to_features_for_prediction' is not defined

In [ ]:
# 예측 함수(데이터 -> 태깅)